#### Integrin

## setup

In [1]:
import os
import sys
import math
import time
import pickle
import copy

#import mdtraj
import MDAnalysis as mda
#import pyemma
#import pyemma.util.contexts
import networkx as nx
import random
#import parmed as pmd
#import acpype
import pandas as pd
from MDAnalysis.analysis import distances
from MDAnalysis.analysis import rms


import numpy as np
from scipy.linalg import eigh
from scipy.spatial.distance import pdist
from scipy.interpolate import interp1d
import matplotlib as mpl
from matplotlib import ticker
if os.environ.get('DISPLAY','') == '':
    print('no display found. Using non-interactive Agg backend')
    mpl.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


import nglview as nv

no display found. Using non-interactive Agg backend


In [2]:

from importlib import reload 
import sys


#### plot settings

In [3]:
showPlots=1
useMagics=1
if useMagics:
    %matplotlib inline
    #%matplotlib notebook
    %load_ext autoreload
    %autoreload 2
    %reload_ext autoreload
    
font = {'weight' : 'normal',
        'size'   : 25}

plt.rc('font', **font)

## load structures

In [4]:


path_to_structures = '/project2/andrewferguson/sivadasetty/doe/analysis-integrin/string_mechanisms/deadbolt/string_parallel_100k_full_lipid_case/final_string/'


cryst_structs=['1tye','1tye_2','1tye_3', '2vc2','2vdk','2vdl','2vdm','2vdn','2vdo','2vdp','2vdq','2vdr',
               '3fcs','3fcs_2','3fcu','3fcu_2','3fcu_3',
               '3nid','3nid_2','3nif','3nif_2','3nig','3nig_2',
               '3t3m','3t3m_2','3t3p','3t3p_2',
               '3zdx','3zdx_2','3zdy','3zdy_2','3zdz','3ze0',
               '3ze1','3ze2','3zdz_2','3ze0_2','3ze1_2','3ze2_2',
               '4z7n','4z7n_2','4z7o','4z7o_2','4z7q','4z7q_2',
               '5hdb','5hdb_2',
               '7tct','7tct_2','7td8','7td8_2','7tho','7tho_2',
               '7tmz','7tmz_2','7tpd','7tpd_2','7u60','7u60_2','7u9f',
               '7u9f_2','7u9v','7u9v_2','7ubr','7ubr_2','7ucy','7ucy_2',
               '7udg','7udg_2','7udh','7udh_2','7ue0','7ue0_2','7ufh',
               '7ufh_2','7uh8','7uh8_2','7uje','7uje_2','7ujk','7ujk_2',
               '7uk9','7uk9_2','7uko','7uko_2','7ukp','7ukp_2','7ukt','7ukt_2']


#'4cak', 
cryo_structs = ['4cak','8gcd', '8gcd_2', '8gce', 
                '8t2u','8t2v', '6v4p', '7la4']

nmr_structs_cryo_likenames=['7l8p','7l8p_2']
          
#'3m10', 
nmr_structs=['1dpk', '1dpq', '1kup','1kuz', '1s4w', '1m8o', 
             '2k1a', '2k9j', '2knc', '2mtp', '2n9y', '7kn0', '7sft']


len(cryst_structs), len(cryo_structs), len(nmr_structs), len (nmr_structs_cryo_likenames)




(89, 8, 13, 2)

##  load RMSD results

In [5]:

rmsd_exp_string_image = {}

for images in range(19):
    

    # Check if file exists
    if os.path.exists(path_to_structures+'/PDBstructures_2/'+'final_string_iter147_rmsd_results_image'+str(images)+'.pkl'):
        with open(path_to_structures+'/PDBstructures_2/'+'final_string_iter147_rmsd_results_image'+str(images)+'.pkl', 'rb') as input_file:     
            rmsd_exp_string_image[images] = pickle.load(input_file)
    
        
    else:
        print(f"Image {images} does not exist")

In [6]:
rmsd_exp_string_image[1]['7ufh_2'],rmsd_exp_string_image[1]['7ufh']

(0.5133864305062278, 0.5282959703886033)

## Create universe for visual analysis

In [9]:
## Crsytal + Cryo snapshots

In [40]:
import glob
from MDAnalysis import transformations
from MDAnalysis.analysis import align

import matplotlib
import matplotlib.cm as cm

import threading

# Generate color values using linspace
# colors = cm.tab20(np.linspace(0, 1, len(alpha_domain_names) + len(beta_domain_names)))

# hexcolors=[]
# for c in np.arange(len(colors)):
#     hexcolors.append(matplotlib.colors.rgb2hex(colors[c]))

# print(hexcolors)
def generate_images(im, filenmae):
#     v.clear()
#     v.add_cartoon(color='red')
#     im0 = v.render_image()
#     v.clear()
#     v.add_cartoon(color='blue')
#     im1 = v.render_image()
    
    #for im in [im0, im1]:
    #while not im.value:
    #    time.sleep(0.1)
        
    with open(filename, 'wb') as fh:
        fh.write(im.value)





hexcolors = [
    "#1f77b4",  # blue
    "#aec7e8",  # light blue
    "#ff7f0e",  # orange
    "#ffbb78",  # light orange
    "#2ca02c",  # green
    "#98df8a",  # light green
    "#d62728",  # red [hyb1]
    "#d62728",  # red [hyb2]
    "#ff9896",  # light red
    "#9467bd",  # purple
    "#c5b0d5",  # light purple
    "#8c564b",  # brown
    "#c49c94",  # light brown
    "#e377c2",  # pink
    "#f7b6d2",  # light pink
    "#7f7f7f",  # gray
    "#c7c7c7",  # light gray
    "#bcbd22",  # olive
    "#dbdb8d",  # light olive
    "#17becf",  # teal
    "#9edae5"   # light teal
]

views_crystal_exp_string = {}
# for string_image in range(19):
    
model=3
# for string_image in [17]:
for string_image in [1]:
    
    
    
    views_crystal_exp_string_pdb = {}
#     for pdbname in exp_structs: #[exp_structs[0]]:
#     for pdbname in ['2vc2','2vdk','2vdm','2vdn','2vdo','2vdp','2vdq','2vdr']: 
    for pdbname in ['7sft']:#,'3fcu_3','3nid_2','3nif_2','3nig_2','3t3m_2','3t3p_2','4z7n_2','4z7o_2','4z7q_2','5hdb_2']: 
        
        
        
        print(string_image, pdbname)

        
        # CA
        #exp_struct = path_to_structures+'/PDBstructures/all_atom_aligned_structures/target_md_'+str(string_image)+'_alignedStructures'+'/'+pdbname+'_aligned_frame_'+str(0)+'.psf'
        #exp_pdb_struct = path_to_structures+'/PDBstructures/all_atom_aligned_structures/target_md_'+str(string_image)+'_alignedStructures'+'/'+pdbname+'_aligned_frame_'+str(0)+'.pdb'
    
        # All atom cryo + cryst
        #exp_struct = path_to_structures+'/PDBstructures_2/all_atom_aligned_structures/target_md_'+str(string_image)+'_alignedStructures'+'/'+pdbname+'_aligned.psf'
        #exp_pdb_struct = path_to_structures+'/PDBstructures_2/all_atom_aligned_structures/target_md_'+str(string_image)+'_alignedStructures'+'/'+pdbname+'_aligned.pdb'
        #exp_universe = mda.Universe(exp_pdb_struct)
        
        # NMR
        exp_struct = path_to_structures+'/PDBstructures_2/all_atom_aligned_structures/target_md_'+str(string_image)+'_alignedStructures'+'/'+pdbname+'_model_'+str(model)+'_aligned.psf'
        exp_pdb_struct = path_to_structures+'/PDBstructures_2/all_atom_aligned_structures/target_md_'+str(string_image)+'_alignedStructures'+'/'+pdbname+'_model_'+str(model)+'_aligned.pdb'
        exp_universe = mda.Universe(exp_pdb_struct)
        
        
        print(path_to_structures+'/PDBstructures_2/all_atom_aligned_structures/target_md_'+str(string_image)+'_alignedStructures'+'/'+pdbname+'_aligned.pdb')
        
        # Rotate if needed
        rotation = transformations.rotate.rotateby(90, direction=[0,0,-1], ag=exp_universe.atoms)
        rotation2 = transformations.rotate.rotateby(270, direction=[0,1,0], ag=exp_universe.atoms)
        #exp_universe.trajectory.add_transformations(rotation,rotation2)
        

        # reducing residue numbers by 1 to show bond connectivity.
        exp_universe_beta_propeller = exp_universe.select_atoms('segid is PROA').atoms.select_atoms('resid 1 to 452')
        exp_universe_thigh = exp_universe.select_atoms('segid is PROA').atoms.select_atoms('resid 452 to 608')
        exp_universe_calf1 = exp_universe.select_atoms('segid is PROA').atoms.select_atoms('resid 608 to 743')
        exp_universe_calf2 = exp_universe.select_atoms('segid is PROA').atoms.select_atoms('resid 743 to 964')
        exp_universe_alpha_tm = exp_universe.select_atoms('segid is PROA').atoms.select_atoms('resid 964 to 1008')

        exp_universe_beta_I = exp_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 109 to 352')
        exp_universe_hyb1 = exp_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 57 to 109')
        exp_universe_hyb2 = exp_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 352 to 432')
        exp_universe_psi = exp_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 1 to 57')
        exp_universe_e1 = exp_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 432 to 475')
        exp_universe_e2 = exp_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 475 to 525')
        exp_universe_e3 = exp_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 525 to 560')
        exp_universe_e4 = exp_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 560 to 604')
        exp_universe_beta_t = exp_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 604 to 690')
        exp_universe_beta_tm = exp_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 690 to 772')

        
        # String CA
        #string_struct = path_to_structures+'/PDBstructures_2/target_md_'+str(string_image)+'_alignedStructures'+'/target_md_'+str(string_image)+'_'+pdbname+'_CA.pdb'
        
        # All atom cryo + cryst
        #string_struct = path_to_structures+'/PDBstructures_2/all_atom_aligned_structures/target_md_'+str(string_image)+'_alignedStructures'+'/target_md_'+str(string_image)+'_'+pdbname+'.pdb'
        
        # NMR
        string_struct = path_to_structures+'/PDBstructures_2/all_atom_aligned_structures/target_md_'+str(string_image)+'_alignedStructures'+'/target_md_'+str(string_image)+'_'+pdbname+'_model_'+str(model)+'.pdb'
        
        string_universe = mda.Universe(string_struct) 
        
        print(path_to_structures+'/PDBstructures_2/all_atom_aligned_structures/target_md_'+str(string_image)+'_alignedStructures'+'/target_md_'+str(string_image)+'_'+pdbname+'.pdb')


        # Translate for visualization
        # Define the translation vector
        rotation = transformations.rotate.rotateby(90, direction=[0,0,-1], ag=string_universe.atoms)
        rotation2 = transformations.rotate.rotateby(270, direction=[0,1,0], ag=string_universe.atoms)
        
        
        
        
        #translation_vector = [-70.0, 0.0, 0.0]  # Translate by 5.0 units in the x-direction
        #translation_vector = [-100.0, 0.0, 0.0]  # Translate by 5.0 units in the x-direction
        #translation_vector = [-110.0, 0.0, 0.0]  # Translate by 5.0 units in the x-direction
    
        # NMR
        #translation_vector = [-15.0, 0.0, 0.0]  # Translate by 5.0 units in the x-direction
        translation_vector = [-20.0, 0.0, 0.0]  # Translate by 5.0 units in the x-direction
        
        translation = transformations.translate(translation_vector)
        
        # Rotate if needed
        #string_universe.trajectory.add_transformations(rotation,rotation2,translation)
        
        # 
        string_universe.trajectory.add_transformations(translation)

        string_universe_beta_propeller = string_universe.select_atoms('segid is PROA').atoms.select_atoms('resid 1 to 452')
        string_universe_thigh = string_universe.select_atoms('segid is PROA').atoms.select_atoms('resid 452 to 608')
        string_universe_calf1 = string_universe.select_atoms('segid is PROA').atoms.select_atoms('resid 608 to 743')
        string_universe_calf2 = string_universe.select_atoms('segid is PROA').atoms.select_atoms('resid 743 to 964')
        string_universe_alpha_tm = string_universe.select_atoms('segid is PROA').atoms.select_atoms('resid 964 to 1008')

        string_universe_beta_I = string_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 109 to 352')
        string_universe_hyb1 = string_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 57 to 109')
        string_universe_hyb2 = string_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 352 to 432')
        string_universe_psi = string_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 1 to 57')
        string_universe_e1 = string_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 432 to 475')
        string_universe_e2 = string_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 475 to 525')
        string_universe_e3 = string_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 525 to 560')
        string_universe_e4 = string_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 560 to 604')
        string_universe_beta_t = string_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 604 to 690')
        string_universe_beta_tm = string_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 690 to 772')
        
        


        # Align u2 onto u1 using the selected atoms
        # aligner = align.AlignTraj(exp_universe_image_dict[17]['2vc2'], exp_universe_image_dict[17]['2vc2'], 
        #                           select='all', in_memory=True)
        # aligner.run()

        #view.clear()
        view = nv.NGLWidget()
        view.clear()


        all_alpha_exp_domains = [exp_universe_beta_propeller, exp_universe_thigh, exp_universe_calf1, 
                                 exp_universe_calf2, exp_universe_alpha_tm]

        all_beta_exp_domains = [exp_universe_beta_I, 
                                exp_universe_hyb1,
                                exp_universe_hyb2,
                                exp_universe_psi, exp_universe_e1, exp_universe_e2,
                                exp_universe_e3, exp_universe_e4, exp_universe_beta_t,
                                exp_universe_beta_tm]

        all_alpha_string_domains = [string_universe_beta_propeller, string_universe_thigh, string_universe_calf1, 
                                  string_universe_calf2, string_universe_alpha_tm]

        all_beta_string_domains = [string_universe_beta_I, 
                                   string_universe_hyb1,
                                   string_universe_hyb2,
                                  string_universe_psi, string_universe_e1, string_universe_e2,
                                  string_universe_e3, string_universe_e4, string_universe_beta_t,
                                  string_universe_beta_tm]




        count_i=0
        count_colors = 0
        for e in all_alpha_exp_domains:

            if e.atoms.n_atoms != 0:
                view.add_trajectory(nv.MDAnalysisTrajectory(e))#, component=count_i)
                view.clear_representations(component=count_i)
                #print(count_colors, count_i, e.atoms.n_atoms, hexcolors[count_colors])
                view.add_representation('cartoon', selection='all', radiusScale=1, component=count_i, 
                                        colorScheme='uniform', color=hexcolors[count_colors])                                
                count_i+=1
            count_colors+=1

        #print(count_i, count_colors)

        for e in all_beta_exp_domains:

            #print(e)
            if e.atoms.n_atoms != 0:
                #print(e.atoms.n_atoms)
                #print(count_colors, count_i, e.atoms.n_atoms, hexcolors[count_colors])
                view.add_trajectory(nv.MDAnalysisTrajectory(e))#, component=count_i)
                view.clear_representations(component=count_i)
                view.add_representation('cartoon', selection='all', radiusScale=1, component=count_i, 
                                        colorScheme='uniform', color=hexcolors[count_colors])
                count_i+=1
            count_colors+=1
    
        #print(count_i, count_colors)
        #print('\n String \n')

        count_colors = 0
        for e in all_alpha_string_domains:

            if e.atoms.n_atoms != 0:
                view.add_trajectory(nv.MDAnalysisTrajectory(e))
                view.clear_representations(component=count_i)
                #print(count_colors, count_i, e.atoms.n_atoms, hexcolors[count_colors])
                view.add_representation('cartoon', selection='all', radiusScale=1, component=count_i, 
                                        colorScheme='uniform', color=hexcolors[count_colors])                                
                count_i+=1
            count_colors+=1

        #print(count_i, count_colors)

        for e in all_beta_string_domains:

            if e.atoms.n_atoms != 0:
                #print(e.atoms.n_atoms)
                #print(count_colors, count_i, e.atoms.n_atoms, hexcolors[count_colors])
                view.add_trajectory(nv.MDAnalysisTrajectory(e))
                view.clear_representations(component=count_i)
                view.add_representation('cartoon', selection='all', radiusScale=1, component=count_i, 
                                        colorScheme='uniform', color=hexcolors[count_colors])
                count_i+=1
            count_colors+=1

        #print(count_i, count_colors)

        
        #view.gui_style = "NGL"
        # Display the widget

        view.control.spin([1, 0, 0], -math.pi/2)


        view._set_size('800px', '800px')

        view.camera = 'orthographic'
        
        #print(view)
        
        views_crystal_exp_string_pdb[pdbname] = view#.render_image()
        
        #view.download_image('integrin_'+pdbname+'_'+str(images)+'.png')
        
#view
    views_crystal_exp_string[string_image] = views_crystal_exp_string_pdb




1 7sft
/project2/andrewferguson/sivadasetty/doe/analysis-integrin/string_mechanisms/deadbolt/string_parallel_100k_full_lipid_case/final_string//PDBstructures_2/all_atom_aligned_structures/target_md_1_alignedStructures/7sft_aligned.pdb
/project2/andrewferguson/sivadasetty/doe/analysis-integrin/string_mechanisms/deadbolt/string_parallel_100k_full_lipid_case/final_string//PDBstructures_2/all_atom_aligned_structures/target_md_1_alignedStructures/target_md_1_7sft.pdb


/project2/andrewferguson/sivadasetty/doe/elss/lib/python3.9/site-packages/MDAnalysis/topology/PDBParser.py:331: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "


In [39]:
# # ! /project2/andrewferguson/sivadasetty/doe/analysis-integrin/string_mechanisms/deadbolt/string_parallel_100k_full_lipid_case/final_string//PDBstructures_2/all_atom_aligned_structures/target_md_14_alignedStructures/
views_crystal_exp_string[1]['7sft']


NGLWidget()

In [453]:

# all_alpha_exp_domains,  all_beta_exp_domains, all_alpha_string_domains, all_beta_string_domains
# 

## Save images

In [41]:
# string_image = 17
# pdbname='2vc2'

for string_image, inner_dict in views_crystal_exp_string.items():
    for pdbname in inner_dict:

        display(views_crystal_exp_string[string_image][pdbname])
        views_crystal_exp_string[string_image][pdbname].download_image(filename='integrin_string_'+str(string_image)+'_'+pdbname+'.png', 
                            factor=10, 
                            antialias=False, trim=True, transparent=True)

# display.clear_output()
        

NGLWidget()

In [15]:
# from IPython import display
# # from IPython.display import clear_output
# for string_image, inner_dict in views_crystal_exp_string.items():
#     #print(string_image,inner_dict)
#     for pdbname in inner_dict:
        
#         #print(pdbname)
        
#         display.display(views_crystal_exp_string[string_image][pdbname])
#         view.download_image(filename='integrin_string_'+str(string_image)+'_'+pdbname+'.png', 
#                             factor=5, 
#                             antialias=False, trim=True, transparent=True)
        
#         display.clear_output()
        

In [13]:
views_crystal_exp_string

{17: {'2vc2': NGLWidget(),
  '2vdk': NGLWidget(n_components=14),
  '2vdm': NGLWidget(),
  '2vdn': NGLWidget(),
  '2vdo': NGLWidget(),
  '2vdp': NGLWidget(),
  '2vdq': NGLWidget(),
  '2vdr': NGLWidget()}}

In [ ]:
# ref_struct = '2vdq'
# string_image = '17'

In [ ]:




# #psf
# # exp_struct = path_to_structures+'/PDBstructures/target_md_'+str(string_image)+'_alignedStructures'+'/'+ref_struct+'_aligned_CA_frame_'+str(0)+'.psf'
# # exp_pdb_struct = path_to_structures+'/PDBstructures/target_md_'+str(string_image)+'_alignedStructures'+'/'+ref_struct+'_aligned_CA_frame_'+str(0)+'.pdb'
# # exp_universe = mda.Universe(exp_pdb_struct)

# exp_struct = path_to_structures+'/PDBstructures/all_atom_aligned_structures/target_md_'+str(string_image)+'_alignedStructures'+'/'+ref_struct+'_aligned_frame_'+str(0)+'.psf'
# exp_pdb_struct = path_to_structures+'/PDBstructures/all_atom_aligned_structures/target_md_'+str(string_image)+'_alignedStructures'+'/'+ref_struct+'_aligned_frame_'+str(0)+'.pdb'
# exp_universe = mda.Universe(exp_pdb_struct)

# print(exp_struct)

# alpha_exp_universe = exp_universe.select_atoms('segid is PROA')
# beta_exp_universe = exp_universe.select_atoms('segid is PROB')

# # reducing residue numbers by 1 to show bond connectivity.
# exp_universe_beta_propeller = exp_universe.select_atoms('segid is PROA').atoms.select_atoms('resid 1 to 452')
# exp_universe_thigh = exp_universe.select_atoms('segid is PROA').atoms.select_atoms('resid 452 to 608')
# exp_universe_calf1 = exp_universe.select_atoms('segid is PROA').atoms.select_atoms('resid 608 to 743')
# exp_universe_calf2 = exp_universe.select_atoms('segid is PROA').atoms.select_atoms('resid 743 to 964')
# exp_universe_alpha_tm = exp_universe.select_atoms('segid is PROA').atoms.select_atoms('resid 964 to 1008')

# exp_universe_beta_I = exp_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 109 to 352')
# exp_universe_hyb1 = exp_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 57 to 109')
# exp_universe_hyb2 = exp_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 352 to 432')
# exp_universe_psi = exp_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 1 to 57')
# exp_universe_e1 = exp_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 432 to 475')
# exp_universe_e2 = exp_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 475 to 525')
# exp_universe_e3 = exp_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 525 to 560')
# exp_universe_e4 = exp_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 560 to 604')
# exp_universe_beta_t = exp_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 604 to 690')
# exp_universe_beta_tm = exp_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 690 to 772')


# #pdb
# # string_struct = path_to_structures+'/PDBstructures/target_md_'+str(string_image)+'_alignedStructures'+'/target_md_'+str(string_image)+'_'+ref_struct+'_CA_frame_'+str(0)+'.pdb'
# # string_universe = mda.Universe(string_struct) 

# string_struct = path_to_structures+'/PDBstructures/all_atom_aligned_structures/target_md_'+str(string_image)+'_alignedStructures'+'/target_md_'+str(string_image)+'_'+ref_struct+'.pdb'
# string_universe = mda.Universe(string_struct) 


# from MDAnalysis import transformations

# # Translate for visualization
# # Define the translation vector
# translation_vector = [-75.0, 0.0, 0.0]  # Translate by 5.0 units in the x-direction
# translation = transformations.translate(translation_vector)
# string_universe.trajectory.add_transformations(translation)
    
# string_universe_beta_propeller = string_universe.select_atoms('segid is PROA').atoms.select_atoms('resid 1 to 452')
# string_universe_thigh = string_universe.select_atoms('segid is PROA').atoms.select_atoms('resid 452 to 608')
# string_universe_calf1 = string_universe.select_atoms('segid is PROA').atoms.select_atoms('resid 608 to 743')
# string_universe_calf2 = string_universe.select_atoms('segid is PROA').atoms.select_atoms('resid 743 to 964')
# string_universe_alpha_tm = string_universe.select_atoms('segid is PROA').atoms.select_atoms('resid 964 to 1008')

# string_universe_beta_I = string_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 109 to 352')
# string_universe_hyb1 = string_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 57 to 109')
# string_universe_hyb2 = string_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 352 to 432')
# string_universe_psi = string_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 1 to 57')
# string_universe_e1 = string_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 432 to 475')
# string_universe_e2 = string_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 475 to 525')
# string_universe_e3 = string_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 525 to 560')
# string_universe_e4 = string_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 560 to 604')
# string_universe_beta_t = string_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 604 to 690')
# string_universe_beta_tm = string_universe.select_atoms('segid is PROB').atoms.select_atoms('resid 690 to 772')





In [ ]:
## Visual check

In [ ]:
# # Domain composition alpha IIb:
# alpha_domain_names = {'beta_propeller': '1-452', 'Thigh': '453-608', 'Calf-1': '609-743',
#                 'Calf-2': '744-964', 'alpha_tm': '965-1008'}
# beta_domain_names = {'betaI': '110-352', 'Hyb1': '58-109', 'Hyb2': '353-432', 'Psi': '1-57',
#                 'E1': '433-475', 'E2': '476-525', 'E3': '526-560',
#                 'E4': '561-604', 'beta-T': '605-690', 'beta_tm': '691-772'}

In [ ]:
# from MDAnalysis.analysis import align

# import matplotlib
# import matplotlib.cm as cm

# # Align u2 onto u1 using the selected atoms
# # aligner = align.AlignTraj(exp_universe_image_dict[17]['2vc2'], exp_universe_image_dict[17]['2vc2'], 
# #                           select='all', in_memory=True)
# # aligner.run()

# view = nv.NGLWidget()



# all_alpha_exp_domains = [exp_universe_beta_propeller, exp_universe_thigh, exp_universe_calf1, 
#                          exp_universe_calf2, exp_universe_alpha_tm]

# all_beta_exp_domains = [exp_universe_beta_I, 
#                         exp_universe_hyb1,
#                         exp_universe_hyb2,
#                         exp_universe_psi, exp_universe_e1, exp_universe_e2,
#                         exp_universe_e3, exp_universe_e4, exp_universe_beta_t,
#                         exp_universe_beta_tm]

# all_alpha_string_domains = [string_universe_beta_propeller, string_universe_thigh, string_universe_calf1, 
#                           string_universe_calf2, string_universe_alpha_tm]

# all_beta_string_domains = [string_universe_beta_I, 
#                            string_universe_hyb1,
#                            string_universe_hyb2,
#                           string_universe_psi, string_universe_e1, string_universe_e2,
#                           string_universe_e3, string_universe_e4, string_universe_beta_t,
#                           string_universe_beta_tm]

# # Generate color values using linspace
# # colors = cm.tab20(np.linspace(0, 1, len(alpha_domain_names) + len(beta_domain_names)))

# # hexcolors=[]
# # for c in np.arange(len(colors)):
# #     hexcolors.append(matplotlib.colors.rgb2hex(colors[c]))

# # print(hexcolors)


# hexcolors = [
#     "#1f77b4",  # blue
#     "#aec7e8",  # light blue
#     "#ff7f0e",  # orange
#     "#ffbb78",  # light orange
#     "#2ca02c",  # green
#     "#98df8a",  # light green
#     "#d62728",  # red [hyb1]
#     "#d62728",  # red [hyb2]
#     "#ff9896",  # light red
#     "#9467bd",  # purple
#     "#c5b0d5",  # light purple
#     "#8c564b",  # brown
#     "#c49c94",  # light brown
#     "#e377c2",  # pink
#     "#f7b6d2",  # light pink
#     "#7f7f7f",  # gray
#     "#c7c7c7",  # light gray
#     "#bcbd22",  # olive
#     "#dbdb8d",  # light olive
#     "#17becf",  # teal
#     "#9edae5"   # light teal
# ]


# count_i=0
# count_colors = 0
# for e in all_alpha_exp_domains:
    
#     if e.atoms.n_atoms != 0:
#         view.add_trajectory(nv.MDAnalysisTrajectory(e))#, component=count_i)
#         view.clear_representations(component=count_i)
#         print(count_colors, count_i, e.atoms.n_atoms, hexcolors[count_colors])
#         view.add_representation('cartoon', selection='all', radiusScale=1, component=count_i, 
#                                 colorScheme='uniform', color=hexcolors[count_colors])                                
#         count_i+=1
#     count_colors+=1
        
# print(count_i, count_colors)

# for e in all_beta_exp_domains:
        
#     #print(e)
#     if e.atoms.n_atoms != 0:
#         #print(e.atoms.n_atoms)
#         print(count_colors, count_i, e.atoms.n_atoms, hexcolors[count_colors])
#         view.add_trajectory(nv.MDAnalysisTrajectory(e))#, component=count_i)
#         view.clear_representations(component=count_i)
#         view.add_representation('cartoon', selection='all', radiusScale=1, component=count_i, 
#                                 colorScheme='uniform', color=hexcolors[count_colors])
#         count_i+=1
#     count_colors+=1
    
# print(count_i, count_colors)

# print('\n String \n')

# count_colors = 0
# for e in all_alpha_string_domains:
    
#     if e.atoms.n_atoms != 0:
#         view.add_trajectory(nv.MDAnalysisTrajectory(e))
#         view.clear_representations(component=count_i)
#         print(count_colors, count_i, e.atoms.n_atoms, hexcolors[count_colors])
#         view.add_representation('cartoon', selection='all', radiusScale=1, component=count_i, 
#                                 colorScheme='uniform', color=hexcolors[count_colors])                                
#         count_i+=1
#     count_colors+=1
        
# print(count_i, count_colors)

# for e in all_beta_string_domains:
        
#     if e.atoms.n_atoms != 0:
#         #print(e.atoms.n_atoms)
#         print(count_colors, count_i, e.atoms.n_atoms, hexcolors[count_colors])
#         view.add_trajectory(nv.MDAnalysisTrajectory(e))
#         view.clear_representations(component=count_i)
#         view.add_representation('cartoon', selection='all', radiusScale=1, component=count_i, 
#                                 colorScheme='uniform', color=hexcolors[count_colors])
#         count_i+=1
#     count_colors+=1
    
# print(count_i, count_colors)

# view.gui_style = "NGL"
# # Display the widget

# view.control.spin([1, 0, 0], -math.pi/2)

# view._set_size('800px', '800px')

# view.download_image('temp.png')
# view


